In [41]:
import torch
import torch.nn.functional as F
import pandas as pd

# Data Preparation

In [21]:
iris_df = pd.read_csv('./data/Iris.csv')
iris_df.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [22]:
X = iris_df.drop(["species"], axis=1).values
y = iris_df["species"].values

Rescale all features to ensure one feature does not dominate the model predictions. In this case we center all features to 0 mean and unit variance.

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=18)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Convert the categorial variable 'y' to one-hot encoding.

In [36]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [37]:
y_train.head(5)

,Iris-setosa,Iris-versicolor,Iris-virginica
0,0,0,1
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1


Convert all variables to tensors.

In [38]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

# Model Definition

In [69]:
class IrisModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(IrisModel, self).__init__()
        self.layer1 = torch.nn.Linear(input_dim, 16)
        self.layer2 = torch.nn.Linear(16, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.softmax(self.layer2(x), dim=1)
        return x

# Model Training

In [70]:
model = IrisModel(X_train.shape[1])
learning_rate = 0.01
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [71]:
no_of_epocs = 1000
train_loss = list()
test_loss = list()

for e in range(no_of_epocs):
    optimizer.zero_grad()
    
    # train
    tr_output = model(X_train)
    tr_loss = criterion(tr_output, y_train.double())
    tr_loss.backward()
    optimizer.step()
    
    # test
    tst_output = model(X_test)
    tst_loss = criterion(tst_output, y_test.double())

    train_loss.append(train_loss)
    test_loss.append(tst_loss)
    
    if e%50 == 0:
        print(f"Epoch No: {e}, Training Loss: {tr_loss}, Testing Loss: {tst_loss}")

Epoch No: 0, Training Loss: 1.1418263415495555, Testing Loss: 1.1446060220400491
Epoch No: 50, Training Loss: 0.6678481668233871, Testing Loss: 0.6539150695006053
Epoch No: 100, Training Loss: 0.5913095250725746, Testing Loss: 0.5813740670681
Epoch No: 150, Training Loss: 0.5811779196063678, Testing Loss: 0.5706147988637288
Epoch No: 200, Training Loss: 0.5770851984620095, Testing Loss: 0.5661000887552897
Epoch No: 250, Training Loss: 0.574740802248319, Testing Loss: 0.5637828807036082
Epoch No: 300, Training Loss: 0.5728683819373449, Testing Loss: 0.5627630194028218
Epoch No: 350, Training Loss: 0.570982250571251, Testing Loss: 0.5634482522805532
Epoch No: 400, Training Loss: 0.5688546086351077, Testing Loss: 0.5668120145797729
Epoch No: 450, Training Loss: 0.5667917708555857, Testing Loss: 0.57186292608579
Epoch No: 500, Training Loss: 0.5650921156009038, Testing Loss: 0.5762700299421947
Epoch No: 550, Training Loss: 0.5638097946842512, Testing Loss: 0.579246457417806
Epoch No: 600, 

# Model Evaluation

In [83]:
from sklearn.metrics import accuracy_score

train_output = model(X_train)
test_output = model(X_test)

correct_train = torch.mean((torch.argmax(y_train, axis=1) == torch.argmax(train_output, axis=1)).double())
correct_test = torch.mean((torch.argmax(y_test, axis=1) == torch.argmax(test_output, axis=1)).double())
print(f"Train Accuracy: {correct_train} Test Accuracy: {correct_test}")

Train Accuracy: 0.9916666666666667 Test Accuracy: 0.9666666666666667
